# Module 6: AI for QEM (SVR & LSTM)

Standard ZNE assumes noise is a simple straight line. Real noise is complex, non-linear, and context-dependent. AI methods—specifically Deep Learning—can capture these complex error patterns.

## 6.1 The Strategy

We will compare two approaches:
1.  **Support Vector Regression (SVR):** A robust classical ML model that predicts error based on summary features (depth, gate count).
2.  **LSTM (Recurrent Neural Network):** A Deep Learning model that reads the circuit gate-by-gate, "remembering" how error accumulates over time.

In [ ]:
import numpy as np
import random
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, thermal_relaxation_error, ReadoutError
from qiskit.quantum_info import Clifford

# --- 1. RE-USE COMPONENTS FROM PREVIOUS MODULES ---
# (We copy them here to make this module self-contained)

def create_random_clifford_circuit(num_qubits, depth):
    # Simple generator for 2-qubit circuits (easier to learn)
    qc = QuantumCircuit(num_qubits)
    gates_1q = ['h', 'x', 'z', 'id'] # 'id' is a wait cycle
    
    for _ in range(depth):
        q = random.randint(0, num_qubits - 1)
        if random.random() > 0.5:
             target = (q + 1) % num_qubits
             qc.cx(q, target)
        else:
             g = random.choice(gates_1q)
             getattr(qc, g)(q)
    return qc

def build_noise_model():
    # Standard noise model
    noise_model = NoiseModel()
    error_1q = thermal_relaxation_error(50e-6, 70e-6, 50e-9)
    error_2q = thermal_relaxation_error(50e-6, 70e-6, 400e-9).expand(thermal_relaxation_error(50e-6, 70e-6, 400e-9))
    noise_model.add_all_qubit_quantum_error(error_1q, ['x', 'h', 'id', 'z'])
    noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])
    return noise_model

def get_expectation_value(qc, simulator):
    qc_t = transpile(qc, simulator)
    qc_t.measure_all()
    result = simulator.run(qc_t, shots=1000).result()
    counts = result.get_counts()
    # Calculate <ZZ> for 2 qubits
    shots = sum(counts.values())
    p_even = (counts.get('00', 0) + counts.get('11', 0)) / shots
    p_odd = (counts.get('01', 0) + counts.get('10', 0)) / shots
    return p_even - p_odd

# --- 2. GENERATE DATASET ---
print("Generating Dataset (this may take 30s)...")
X_features = [] # [depth, cx_count]
y_noisy = []    # Input to AI
y_ideal = []    # Target for AI
y_error = []    # Residual (Ideal - Noisy)

sim_ideal = AerSimulator(method='stabilizer')
sim_noisy = AerSimulator(noise_model=build_noise_model())

for _ in range(200):
    depth = random.randint(5, 50)
    qc = create_random_clifford_circuit(2, depth)
    
    # Calculate Values
    # Ideally, we calculate ideal using Clifford() object, but simulator is fine for demo
    val_ideal = get_expectation_value(qc.copy(), sim_ideal)
    val_noisy = get_expectation_value(qc.copy(), sim_noisy)
    
    # Extract Features
    ops = qc.count_ops()
    feats = [depth, ops.get('cx', 0)]
    
    X_features.append(feats)
    y_noisy.append(val_noisy)
    y_ideal.append(val_ideal)
    y_error.append(val_ideal - val_noisy)

X_train, X_test, y_err_train, y_err_test, y_noisy_train, y_noisy_test = train_test_split(
    X_features, y_error, y_noisy, test_size=0.2
)
print(f"Dataset generated. Train size: {len(X_train)}")

## 6.2 Approach A: Support Vector Regression (SVR)

We train an SVR to predict the **Error** ($Ideal - Noisy$) based on circuit features.

In [ ]:
# Train SVR
svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.01))
svr.fit(X_train, y_err_train)
print("SVR Trained.")

# Evaluate
predicted_errors = svr.predict(X_test)
final_predictions = np.array(y_noisy_test) + predicted_errors
true_ideals = np.array(y_noisy_test) + np.array(y_err_test)

mse = np.mean((final_predictions - true_ideals)**2)
print(f"SVR Mean Squared Error: {mse:.4f}")

## 6.3 Approach B: LSTM (Neural Network)

We define a simple LSTM that takes the circuit features and learns to predict the error correction.

In [ ]:
# Convert data to PyTorch Tensors
# LSTM expects input shape (Batch, Sequence_Length, Features)
# Since we are using summary features (not gate sequences yet), 
# we treat this as a sequence of length 1 for simplicity in this demo module.

X_train_t = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train_t = torch.tensor(y_err_train, dtype=torch.float32).unsqueeze(1)
X_test_t = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

class QEM_LSTM(nn.Module):
    def __init__(self, input_size=2, hidden_size=16):
        super(QEM_LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # Take the last output step
        last_step = lstm_out[:, -1, :]
        return self.fc(last_step)

model = QEM_LSTM()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("Training LSTM...")
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Evaluate LSTM
with torch.no_grad():
    lstm_preds = model(X_test_t).squeeze().numpy()

lstm_final = np.array(y_noisy_test) + lstm_preds
lstm_mse = np.mean((lstm_final - true_ideals)**2)
print(f"LSTM Mean Squared Error: {lstm_mse:.4f}")

print("\n--- CONCLUSION ---")
if lstm_mse < mse:
    print("LSTM outperformed SVR! (Deep learning captured the complexity)")
else:
    print("SVR outperformed LSTM! (Sometimes simpler is better for small data)")